In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('company.db')

In [4]:
c = conn.cursor()
# c.execute("""CREATE TABLE company_details(
#               company_url VARCHAR(5000),
#               year int
             
#         )""")
conn.commit()
# conn.close()
# c.execute("drop table company_details")

In [5]:
import pandas as pd
df = pd.read_csv(r'E:\pdf_details.csv')

In [6]:
df

,company_url,latest_year_of_pdf
0,http://www.adidas-group.com/en/investors/corpo...,2019
1,https://www.allianz.com/en/about_us/management...,2019
2,https://www.basf.com/global/en/investors/basf-...,2019
3,http://www.bayer.com/en/Corporate-Governance.aspx,2019
4,https://www.bmwgroup.com/en/company/company-po...,2019
5,https://www.continental-corporation.com/en/com...,2019
6,https://www.covestro.com/en/company/management...,2019
7,https://www.daimler.com/company/corporate-gove...,2019
8,https://ir.deliveryhero.com/websites/delivery/...,2019
9,https://www.db.com/ir/en/documents.htm,2019


In [24]:
df.to_sql('company_details', conn, if_exists='replace', index = False)

In [25]:
pd.read_sql("select * from company_details", conn)

,company_url,latest_year_of_pdf
0,http://www.adidas-group.com/en/investors/corpo...,2019
1,https://www.allianz.com/en/about_us/management...,2020
2,https://www.basf.com/global/en/investors/basf-...,2020
3,http://www.bayer.com/en/Corporate-Governance.aspx,2019
4,https://www.bmwgroup.com/en/company/company-po...,2019
5,https://www.continental-corporation.com/en/com...,2019
6,https://www.covestro.com/en/company/management...,2020
7,https://www.daimler.com/company/corporate-gove...,2020
8,https://ir.deliveryhero.com/websites/delivery/...,2020
9,https://www.db.com/ir/en/documents.htm,2020


In [16]:
import requests
import re
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

url = 'https://www.dcgk.de/en/declarations-of-compliance.html'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
text=[]
for link in soup.find_all('a'):
    text.append(link)

c_url=[]
for i in text:
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(i))
    c_url.append(urls)
res = [ele for ele in c_url if ele != []]
flatList = [ item for elem in res for item in elem]
del flatList[0:4]
flatList = flatList[0:11]

In [17]:
import os
import requests
import pandas as pd  
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [21]:
def get_lates_pdf_from_link(url_list,db_years_list, keyword_list, file_path):
    
    if not os.path.exists(file_path):
        os.mkdir(folder_location)
    C = db_years_list.copy()
    for url,db_year,year_loc in zip(url_list,db_years_list,range(len(url_list))):
        try:
            response = requests.get(url, timeout=4)
            soup= BeautifulSoup(response.text, "html.parser")
            test = 1
        except:
            print("Not able to load following url:\n", url, "\n\n")
            test = 0
    
        if test == 1:    
            try:
                df_chs = pd.DataFrame(columns = ["year", "link"])
                for key in keyword_list:
                    links = soup.select("a[href*='%s']"%key)
                    if len(links) > 0:
                        for link in links:
                            #Name the pdf files using the last portion of each link which are unique in this case
                            last_links = link['href'].split('/')[-1]
                            for split_char in ['_','(',')','-', '.']:
                                last_links = ' '.join(last_links.split(split_char))
                            year = [int(num) for num in last_links.split() if (num.isdigit()) & (len(num) == 4)]
                            if len(year) > 0:
                                df_chs.loc[len(df_chs)] = [year[0], link['href']]
                if len(df_chs) > 0:
                    df_chs.sort_values("year", axis=0, ascending=False, inplace=True)
#                     print(df_chs.head(10))
                    pdf_year = int(df_chs.iloc[0,0])    
#                     print(pdf_year)
                    if int(db_year) < pdf_year:
                        filename = str(df_chs.iloc[0,1])
#                         print(filename)
                        filename = filename.split("/")[-1].split(".pdf")[0] + ".pdf"
#                         print(filename)
                        link = str(df_chs.iloc[0,1])
    #                     print(link)
                        C[year_loc] = pdf_year
                        
                    else:
                        filename = ''
                else:
                    filename = ''
            except TypeError:
                filename = ''

            if len(filename) > 5:
                with open(folder_location+filename, 'wb') as f:
                    f.write(requests.get(urljoin(url,link)).content)
                    f.close()
    return C


In [19]:
df1 = pd.read_csv(r"D:\Book1.csv", encoding='unicode_escape')
keywords = df1["keywords"].dropna().tolist()
#url_list = df["company_link"].dropna().tolist()
#url_list = flatList[0:11]
#url_list = ['https://www.fraport.com/en/investors/corporate-governance.html']
folder_location = r"D:/New_folder/New folder/"

In [20]:
url_list = df['company_url'].tolist()
db_year_list = df['latest_year_of_pdf'].tolist()
years = get_lates_pdf_from_link(url_list,db_year_list, keywords, folder_location)
df['latest_year_of_pdf'] = years

Not able to load following url:
 http://www.adidas-group.com/en/investors/corporate-governance/declaration-compliance/ 


2020
2020
2020
2020
2020
2020
2012


In [22]:
for i in years:
    df.replace(to_replace ="latest_year_of_pdf",
                 value = i)

In [23]:
print(df)

                                          company_url  latest_year_of_pdf
0   http://www.adidas-group.com/en/investors/corpo...                2019
1   https://www.allianz.com/en/about_us/management...                2020
2   https://www.basf.com/global/en/investors/basf-...                2020
3   http://www.bayer.com/en/Corporate-Governance.aspx                2019
4   https://www.bmwgroup.com/en/company/company-po...                2019
5   https://www.continental-corporation.com/en/com...                2019
6   https://www.covestro.com/en/company/management...                2020
7   https://www.daimler.com/company/corporate-gove...                2020
8   https://ir.deliveryhero.com/websites/delivery/...                2020
9              https://www.db.com/ir/en/documents.htm                2020
10  http://deutsche-boerse.com/dbg/dispatch/en/kir...                2019


In [14]:
pd.read_sql("select * from company_details", conn)

,company_url,latest_year_of_pdf
0,http://www.adidas-group.com/en/investors/corpo...,2019
1,https://www.allianz.com/en/about_us/management...,2019
2,https://www.basf.com/global/en/investors/basf-...,2019
3,http://www.bayer.com/en/Corporate-Governance.aspx,2019
4,https://www.bmwgroup.com/en/company/company-po...,2019
5,https://www.continental-corporation.com/en/com...,2019
6,https://www.covestro.com/en/company/management...,2019
7,https://www.daimler.com/company/corporate-gove...,2019
8,https://ir.deliveryhero.com/websites/delivery/...,2019
9,https://www.db.com/ir/en/documents.htm,2019


In [37]:
db_year_list

[2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019]

In [35]:
years

[2019, 2020, 2020, 2019, 2019, 2019, 2020, 2020, 2020, 2020, 2019]

In [36]:
df['latest_year_of_pdf']

0     2019
1     2020
2     2020
3     2019
4     2019
5     2019
6     2020
7     2020
8     2020
9     2020
10    2019
Name: latest_year_of_pdf, dtype: int64

In [ ]:
df = pd.read_csv(r"D:\New_folder\pdf\keywords.csv", encoding='unicode_escape')
keywords = df["keywords"].dropna().tolist()
#url_list = df["company_link"].dropna().tolist()
#url_list = flatList[0:11]
#url_list = ['https://www.fraport.com/en/investors/corporate-governance.html']
folder_location = r"D:/New_folder/New folder/"

In [ ]:
get_lates_pdf_from_link(url_list, keywords, folder_location)

In [9]:
c.execute("INSERT INTO company_details VALUES('https://www.allianz.com/en/about-us/corporate-management/corporate-governance/declaration-of-conformity.html','declaration-of-conformity','2021-04-27')")

In [10]:
c.execute("INSERT INTO company_details VALUES('https://www.basf.com/global/en/investors/basf-at-a-glance/corporate-governance/declarations.html','Declaration_of_Conformity','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.bayer.com/en/corporate-governance/links-and-downloads','Bayer-corporate-governance','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.bmwgroup.com/en/company/company-portrait.html','Declaration-of-Compliance','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.continental.com/en/company/corporate-governance/principles-and-declarations/corporate-governance-principles-13402','Current-declaration-of-continental','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.covestro.com/en/company/management/corporate-governance','Dcgk-delaration-of-conformity','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.daimler.com/company/corporate-governance/declarations/','daimler-entsprechenserklaerung-en','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://ir.deliveryhero.com/websites/delivery/English/4500/declaration-of-compliance.html','Declaration_of_Compliance','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.db.com/ir/en/documents.htm','Declaration_of_Conformity','2021-04-27')")
c.execute("INSERT INTO company_details VALUES('https://www.deutsche-boerse.com/dbg-en/investor-relations/corporate-governance/declaration-of-conformity','entsprechungserklaerung','2021-04-27')")

In [12]:
import pandas as pd
pd.read_sql("select * from company_details", conn)

,company_url,keywords,date_of_download
0,https://www.allianz.com/en/about-us/corporate-...,declaration-of-conformity,2021-04-27
1,https://www.basf.com/global/en/investors/basf-...,Declaration_of_Conformity,2021-04-27
2,https://www.bayer.com/en/corporate-governance/...,Bayer-corporate-governance,2021-04-27
3,https://www.bmwgroup.com/en/company/company-po...,Declaration-of-Compliance,2021-04-27
4,https://www.continental.com/en/company/corpora...,Current-declaration-of-continental,2021-04-27
5,https://www.covestro.com/en/company/management...,Dcgk-delaration-of-conformity,2021-04-27
6,https://www.daimler.com/company/corporate-gove...,daimler-entsprechenserklaerung-en,2021-04-27
7,https://ir.deliveryhero.com/websites/delivery/...,Declaration_of_Compliance,2021-04-27
8,https://www.db.com/ir/en/documents.htm,Declaration_of_Conformity,2021-04-27
9,https://www.deutsche-boerse.com/dbg-en/investo...,entsprechungserklaerung,2021-04-27


In [9]:
df = pd.read_sql("select * from company_details", conn)

In [5]:
import pandas as pd

In [10]:
df.to_sql(conn, 'new_tab')


ArgumentError: Could not parse rfc1738 URL from string 'new_tab'

In [14]:
df.to_sql('company_details', conn, if_exists='replace', index = False)

In [18]:
pd.read_sql("select * from company_details", conn)

,company_url,keywords,date_of_download,updated_date
0,https://www.allianz.com/en/about-us/corporate-...,declaration-of-conformity,2021-04-22,2021-04-23
